# 发送email与接收email
## 发送email
SMTP是发送邮件的协议，Python内置对SMTP的支持，可以发送纯文本邮件、HTML邮件以及带附件的邮件。
Python对SMTP支持有smtplib和email两个模块，email负责构造邮件，smtplib负责发送邮件。

首先，我们来构造一个最简单的纯文本邮件：

In [1]:
from email.mime.text import MIMEText
msg = MIMEText('hello, send by Python...', 'plain', 'utf-8')
msg

注意到构造MIMEText对象时，第一个参数就是邮件正文，第二个参数是MIME的subtype，传入'plain'表示纯文本，最终的MIME就是'text/plain'，最后一定要用utf-8编码保证多语言兼容性。

然后，通过SMTP发出去：

In [9]:
# 输入Email地址和口令:
from_addr = 'hoolich@163.com'#input('From: ')
password = 'FYIRMDFDJGHFIOIA'#input('Password: ')授权码不是邮箱密码
# 输入收件人地址:
to_addr = 'hoolich@163.com'#input('To: ')
# 输入SMTP服务器地址:
smtp_server = 'smtp.163.com'#input('SMTP server: ')

import smtplib
server = smtplib.SMTP(smtp_server, 25) # SMTP协议默认端口是25
server.set_debuglevel(1)
server.login(from_addr, password)
server.sendmail(from_addr, [to_addr], msg.as_string())
server.quit()

send: 'ehlo chenqiangdeMacBook-Pro.local\r\n'
reply: b'250-mail\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-AUTH LOGIN PLAIN XOAUTH2\r\n'
reply: b'250-AUTH=LOGIN PLAIN XOAUTH2\r\n'
reply: b'250-coremail 1Uxr2xKj7kG0xkI17xGrU7I0s8FY2U3Uj8Cz28x1UUUUU7Ic2I0Y2Urk-CIYUCa0xDrUUUUj\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-ID\r\n'
reply: b'250 8BITMIME\r\n'
reply: retcode (250); Msg: b'mail\nPIPELINING\nAUTH LOGIN PLAIN XOAUTH2\nAUTH=LOGIN PLAIN XOAUTH2\ncoremail 1Uxr2xKj7kG0xkI17xGrU7I0s8FY2U3Uj8Cz28x1UUUUU7Ic2I0Y2Urk-CIYUCa0xDrUUUUj\nSTARTTLS\nID\n8BITMIME'
send: 'AUTH PLAIN AGhvb2xpY2hAMTYzLmNvbQBGWUlSTURGREpHSEZJT0lB\r\n'
reply: b'235 Authentication successful\r\n'
reply: retcode (235); Msg: b'Authentication successful'
send: 'mail FROM:<hoolich@163.com>\r\n'
reply: b'250 Mail OK\r\n'
reply: retcode (250); Msg: b'Mail OK'
send: 'rcpt TO:<hoolich@163.com>\r\n'
reply: b'250 Mail OK\r\n'
reply: retcode (250); Msg: b'Mail OK'
send: 'data\r\n'
reply: b'354 End data with <CR><LF>.<CR><LF>

(221, b'Bye')

仔细观察，发现如下问题：

邮件没有主题；
收件人的名字没有显示为友好的名字，比如Mr Green <green@example.com>；
明明收到了邮件，却提示不在收件人中。
这是因为邮件主题、如何显示发件人、收件人等信息并不是通过SMTP协议发给MTA，而是包含在发给MTA的文本中的，所以，我们必须把From、To和Subject添加到MIMEText中，才是一封完整的邮件：

In [10]:
from email import encoders
from email.header import Header
from email.mime.text import MIMEText
from email.utils import parseaddr, formataddr

import smtplib
    
def _format_addr(s):
    name, addr = parseaddr(s)
    return formataddr((Header(name, 'utf-8').encode(), addr))

# 输入Email地址和口令:
from_addr = 'hoolich@163.com'#input('From: ')
password = 'FYIRMDFDJGHFIOIA'#input('Password: ')授权码不是邮箱密码
# 输入收件人地址:
to_addr = 'hoolich@163.com'#input('To: ')
# 输入SMTP服务器地址:
smtp_server = 'smtp.163.com'#input('SMTP server: ')

msg = MIMEText('hello, send by Python...', 'plain', 'utf-8')#内容
msg['From'] = _format_addr('Python爱好者 <%s>' % from_addr)#发件人
msg['To'] = _format_addr('管理员 <%s>' % to_addr)#收件人
msg['Subject'] = Header('来自SMTP的问候……', 'utf-8').encode()#邮件标题
    
server = smtplib.SMTP(smtp_server, 25)
server.set_debuglevel(1)
server.login(from_addr, password)
server.sendmail(from_addr, [to_addr], msg.as_string())
server.quit()

send: 'ehlo chenqiangdeMacBook-Pro.local\r\n'
reply: b'250-mail\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-AUTH LOGIN PLAIN XOAUTH2\r\n'
reply: b'250-AUTH=LOGIN PLAIN XOAUTH2\r\n'
reply: b'250-coremail 1Uxr2xKj7kG0xkI17xGrU7I0s8FY2U3Uj8Cz28x1UUUUU7Ic2I0Y2UFeB1lBUCa0xDrUUUUj\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-ID\r\n'
reply: b'250 8BITMIME\r\n'
reply: retcode (250); Msg: b'mail\nPIPELINING\nAUTH LOGIN PLAIN XOAUTH2\nAUTH=LOGIN PLAIN XOAUTH2\ncoremail 1Uxr2xKj7kG0xkI17xGrU7I0s8FY2U3Uj8Cz28x1UUUUU7Ic2I0Y2UFeB1lBUCa0xDrUUUUj\nSTARTTLS\nID\n8BITMIME'
send: 'AUTH PLAIN AGhvb2xpY2hAMTYzLmNvbQBGWUlSTURGREpHSEZJT0lB\r\n'
reply: b'235 Authentication successful\r\n'
reply: retcode (235); Msg: b'Authentication successful'
send: 'mail FROM:<hoolich@163.com>\r\n'
reply: b'250 Mail OK\r\n'
reply: retcode (250); Msg: b'Mail OK'
send: 'rcpt TO:<hoolich@163.com>\r\n'
reply: b'250 Mail OK\r\n'
reply: retcode (250); Msg: b'Mail OK'
send: 'data\r\n'
reply: b'354 End data with <CR><LF>.<CR><LF>

(221, b'Bye')

## 发送附件
如果Email中要加上附件怎么办？带附件的邮件可以看做包含若干部分的邮件：文本和各个附件本身，所以，可以构造一个MIMEMultipart对象代表邮件本身，然后往里面加上一个MIMEText作为邮件正文，再继续往里面加上表示附件的MIMEBase对象即可：

In [10]:
from email import encoders
from email.header import Header
from email.mime.text import MIMEText
from email.utils import parseaddr, formataddr
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase

import smtplib
    
def _format_addr(s):
    name, addr = parseaddr(s)
    return formataddr((Header(name, 'utf-8').encode(), addr))

# 输入Email地址和口令:
from_addr = 'hoolich@163.com'#input('From: ')
password = 'FYIRMDFDJGHFIOIA'#input('Password: ')授权码不是邮箱密码
# 输入收件人地址:
to_addr = 'hoolich@163.com'#input('To: ')
# 输入SMTP服务器地址:
smtp_server = 'smtp.163.com'#input('SMTP server: ')
msg = MIMEMultipart()#创建一个可以带附件的正文
# msg = MIMEText('hello, send by Python...', 'plain', 'utf-8')#内容
msg['From'] = _format_addr('Python爱好者 <%s>' % from_addr)#发件人
msg['To'] = _format_addr('管理员 <%s>' % to_addr)#收件人
msg['Subject'] = Header('来自SMTP的问候……', 'utf-8').encode()#邮件标题

# 邮件正文是MIMEText:
msg.attach(MIMEText('send with file...', 'plain', 'utf-8'))

# 添加附件就是加上一个MIMEBase，从本地读取一个图片:
with open('/Users/chenqiang/Documents/the-craft-of-selfteaching/images/archimedes-eureka.png', 'rb') as f:
    # 设置附件的MIME和文件名，这里是png类型:
    mime = MIMEBase('image', 'png', filename='archimedes-eureka.png')
    # 加上必要的头信息:
    mime.add_header('Content-Disposition', 'attachment', filename='archimedes-eureka.png')
    mime.add_header('Content-ID', '<0>')
    mime.add_header('X-Attachment-Id', '0')
    # 把附件的内容读进来:
    mime.set_payload(f.read())
    # 用Base64编码:
    encoders.encode_base64(mime)
    # 添加到MIMEMultipart:
    msg.attach(mime)
    
server = smtplib.SMTP(smtp_server, 25)
server.set_debuglevel(1)
server.login(from_addr, password)
server.sendmail(from_addr, [to_addr], msg.as_string())
server.quit()


send: 'ehlo chenqiangdeMacBook-Pro.local\r\n'
reply: b'250-mail\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-AUTH LOGIN PLAIN XOAUTH2\r\n'
reply: b'250-AUTH=LOGIN PLAIN XOAUTH2\r\n'
reply: b'250-coremail 1Uxr2xKj7kG0xkI17xGrU7I0s8FY2U3Uj8Cz28x1UUUUU7Ic2I0Y2UFSlqqpUCa0xDrUUUUj\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-ID\r\n'
reply: b'250 8BITMIME\r\n'
reply: retcode (250); Msg: b'mail\nPIPELINING\nAUTH LOGIN PLAIN XOAUTH2\nAUTH=LOGIN PLAIN XOAUTH2\ncoremail 1Uxr2xKj7kG0xkI17xGrU7I0s8FY2U3Uj8Cz28x1UUUUU7Ic2I0Y2UFSlqqpUCa0xDrUUUUj\nSTARTTLS\nID\n8BITMIME'
send: 'AUTH PLAIN AGhvb2xpY2hAMTYzLmNvbQBGWUlSTURGREpHSEZJT0lB\r\n'
reply: b'235 Authentication successful\r\n'
reply: retcode (235); Msg: b'Authentication successful'
send: 'mail FROM:<hoolich@163.com>\r\n'
reply: b'250 Mail OK\r\n'
reply: retcode (250); Msg: b'Mail OK'
send: 'rcpt TO:<hoolich@163.com>\r\n'
reply: b'250 Mail OK\r\n'
reply: retcode (250); Msg: b'Mail OK'
send: 'data\r\n'
reply: b'354 End data with <CR><LF>.<CR><LF>

(221, b'Bye')

## 收取邮件
SMTP用于发送邮件，如果要收取邮件呢？

收取邮件就是编写一个MUA作为客户端，从MDA把邮件获取到用户的电脑或者手机上。收取邮件最常用的协议是POP协议，目前版本号是3，俗称POP3。

Python内置一个poplib模块，实现了POP3协议，可以直接用来收邮件。

注意到POP3协议收取的不是一个已经可以阅读的邮件本身，而是邮件的原始文本，这和SMTP协议很像，SMTP发送的也是经过编码后的一大段文本。

要把POP3收取的文本变成可以阅读的邮件，还需要用email模块提供的各种类来解析原始文本，变成可阅读的邮件对象。

所以，收取邮件分两步：

>第一步：用poplib把邮件的原始文本下载到本地；

>第二部：用email解析原始文本，还原为邮件对象。

通过POP3下载邮件
POP3协议本身很简单，以下面的代码为例，我们来获取最新的一封邮件内容：

In [3]:
import poplib
from email.parser import Parser
from email.header import decode_header
from email.utils import parseaddr

# 输入邮件地址, 口令和POP3服务器地址:
email = 'hoolich@163.com'
password = 'FYIRMDFDJGHFIOIA'
pop3_server = 'pop3.163.com'

def guess_charset(msg):
    charset = msg.get_charset()
    if charset is None:
        content_type = msg.get('Content-Type', '').lower()
        pos = content_type.find('charset=')
        if pos >= 0:
            charset = content_type[pos + 8:].strip()
    return charset

def decode_str(s):
    value, charset = decode_header(s)[0]
    if charset:
        value = value.decode(charset)
    return value

def print_info(msg, indent=0):
    if indent == 0:
        for header in ['From', 'To', 'Subject']:
            value = msg.get(header, '')
            if value:
                if header=='Subject':
                    value = decode_str(value)
                else:
                    hdr, addr = parseaddr(value)
                    name = decode_str(hdr)
                    value = u'%s <%s>' % (name, addr)
            print('%s%s: %s' % ('  ' * indent, header, value))
    if (msg.is_multipart()):
        parts = msg.get_payload()
        for n, part in enumerate(parts):
            print('%spart %s' % ('  ' * indent, n))
            print('%s--------------------' % ('  ' * indent))
            print_info(part, indent + 1)
    else:
        content_type = msg.get_content_type()
        if content_type=='text/plain' or content_type=='text/html':
            content = msg.get_payload(decode=True)
            charset = guess_charset(msg)
            if charset:
                content = content.decode(charset)
            print('%sText: %s' % ('  ' * indent, content + '...'))
        else:
            print('%sAttachment: %s' % ('  ' * indent, content_type))

# 连接到POP3服务器:
server = poplib.POP3(pop3_server)
# 可以打开或关闭调试信息:
server.set_debuglevel(1)
# 可选:打印POP3服务器的欢迎文字:
print(server.getwelcome().decode('utf-8'))
# 身份认证:
server.user(email)
server.pass_(password)
# stat()返回邮件数量和占用空间:
print('Messages: %s. Size: %s' % server.stat())
# list()返回所有邮件的编号:
resp, mails, octets = server.list()
# 可以查看返回的列表类似[b'1 82923', b'2 2184', ...]
print(mails)
# 获取最新一封邮件, 注意索引号从1开始:
index = len(mails)
resp, lines, octets = server.retr(index)
# lines存储了邮件的原始文本的每一行,
# 可以获得整个邮件的原始文本:
msg_content = b'\r\n'.join(lines).decode('utf-8')
# 稍后解析出邮件:
msg = Parser().parsestr(msg_content)
print_info(msg)
# 可以根据邮件索引号直接从服务器删除邮件:
# server.dele(index)
# 关闭连接:
server.quit()

+OK Welcome to coremail Mail Pop3 Server (163coms[10774b260cc7a37d26d71b52404dcf5cs])
*cmd* 'USER hoolich@163.com'
*cmd* 'PASS FYIRMDFDJGHFIOIA'
*cmd* 'STAT'
*stat* [b'+OK', b'11', b'447684']
Messages: 11. Size: 447684
*cmd* 'LIST'
[b'1 15154', b'2 709', b'3 851', b'4 22603', b'5 14917', b'6 867', b'7 865', b'8 860', b'9 860', b'10 375022', b'11 14976']
*cmd* 'RETR 11'
From: Longbridge 长桥 <noreply@longbridgeapp.com>
To:  <hoolich@163.com>
Subject: LongBridge V4.19.0 更新来袭，抢先了解功能
Text: <!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">
  <head>
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8"/>
    <meta name="viewport" content="width=device-width, initial-scale=1.0"/>
    <style>
      .footer{width:100%;margin:20px auto 0}.footer .end{padding-bottom:20px;margin-bottom:20px;font-size:16px;line-height:24px;color:#1f2a32;border-bottom:1px solid #eaebec}

b'+OK core mail'